In [2]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 50

In [5]:
inputs = tf.keras.layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))

#contractive path
c1 = tf.keras.layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c1)

c2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(c1)
c2 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c2)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c2)

c3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(c2)
c3 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c3)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c3)

c4 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(c3)
c4 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c4)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c4)

c5 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(c4)
c5 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c5)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c5) #shape 8x8x256

#expansion path
c6 = tf.keras.layers.Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(c5)
c6 = tf.keras.layers.concatenate([c4,c6])
c6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c6)

c7 = tf.keras.layers.Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(c6)
c7 = tf.keras.layers.concatenate([c3,c7])
c7 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c7)

c8 = tf.keras.layers.Conv2DTranspose(32,(2,2),strides=(2,2),padding='same')(c7)
c8 = tf.keras.layers.concatenate([c2,c8])
c8 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c8)

c9 = tf.keras.layers.Conv2DTranspose(16,(2,2),strides=(2,2),padding='same')(c8)
c9 = tf.keras.layers.concatenate([c1,c9])
c9 = tf.keras.layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c9)

outputs =  tf.keras.layers.Conv2D(1,(1,1),activation='sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs],outputs=[outputs])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_12 (Conv2D)          (None, 128, 128, 16)         448       ['input_3[0][0]']             
                                                                                                  
 dropout_6 (Dropout)         (None, 128, 128, 16)         0         ['conv2d_12[0][0]']           
                                                                                                  
 conv2d_13 (Conv2D)          (None, 128, 128, 16)         2320      ['dropout_6[0][0]']           
                                                                                              

In [ ]:
#checkpoints and callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5',monitor='val_loss',verbose=1,save_best_only=True)
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='logs'),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2),
             checkpoint]

In [ ]:
# prompt: Write a summary of this notebook

This notebook demonstrates the implementation of a U-Net architecture for image segmentation using TensorFlow and Keras. The U-Net model is a popular deep learning architecture for biomedical image segmentation tasks.

The notebook begins by defining the input and output shapes of the U-Net model. The model consists of a contracting path and an expansive path. The contracting path consists of four max pooling layers and four convolutional blocks. Each convolutional block includes two convolutional layers with a dropout layer in between. The expansive path consists of four upsampling layers and four convolutional blocks. Each upsampling layer is followed by a concatenation with the corresponding layer in the contracting path. The final layer is a 1x1 convolution layer with a sigmoid activation function to predict the segmentation mask.

The model is compiled with the Adam optimizer and the binary cross-entropy loss function. The model summary is printed to show the number of parameters and the shapes of the layers.

The notebook also defines a checkpoint callback to save the model with the best validation loss. The model is then trained on the provided dataset using the `fit` method.

Finally, the notebook provides a brief summary of the U-Net architecture and its application in biomedical image segmentation.